In [ ]:
import pandas as pd
import numpy as np

from sklearn.decomposition import PCA
from matplotlib import pyplot as plt

### Reading in the data

For this analysis we have our data already prepared. `prod_sales` will be a data frame with 1001 columns and 2674 rows. Each row is an owner and the owner number is in the first column. The remaining 1000 columns are the purchases by that owner across the top 1000 products. Let's read the data in and look at the total sales for the top 20 products.

In [ ]:
prod_sales = pd.read_csv("owner_level_top_prod_sales.txt",sep="\t")

In [ ]:
prod_sales

In [ ]:
product_summary = prod_sales.iloc[:,1:].sum().reset_index()
product_summary = product_summary.rename(columns={"index":"product",0:"total_sales"})
product_summary = product_summary.set_index("product")


In [ ]:
product_summary.sort_values(by="total_sales").tail(n=10)

In [ ]:
product_summary.sort_values(by='total_sales').tail(n=10).plot(kind="bar")
plt.title("Total Sales by Product")
plt.ylabel("Sales across All Owners")
plt.show()

### PCA on Wedge product sales

These data represent a typical use case for PCA, with our 1000 dimensions of spend data for each owner. Let's fit a PCA model on the product column. We'll use the PCA function from sci-kit learn. You can learn much more about the functionality by reading the [documentation](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html). 

In [ ]:
pca = PCA()

In [ ]:
pca.fit(prod_sales.drop(columns="owner"))

Typically the first thing we do when working with PCA output is to look at the explained variance. Our PCA object has two variance statistics: `explained_variance_` and `explained_variance_ratio_`. The former is the variance explained by the component in raw terms, the latter normalizes by the total variance of the data set. Let's look at the normalized value for the first 20 components.

In [ ]:
explained_variance = pd.DataFrame(pca.explained_variance_ratio_)
#showing the first value
ax = explained_variance.head(20).plot.bar(legend=False, figsize=(8, 4))
ax.set_xlabel('Component')
plt.tight_layout()
plt.show()

We can see that the first component explains a much larger portion of the variance than any other component. The next two explain more than the fourth. After the fifth component we see a general flattening of the curve, at least at this scale. 

It can also be nice to look at cumulative variance over a larger range of components. 

In [ ]:
cume_explained_variance = pd.DataFrame(pca.explained_variance_ratio_).cumsum()

In [ ]:
#showing the first value
ax = cume_explained_variance.head(150).plot.line(legend=False, figsize=(8, 4))
ax.set_xlabel('Component')
plt.tight_layout()
plt.show()

As we can see, you need around 150 components to get to 90% variance explained. There's nothing magic about 90%, although it's often a convenient cutoff when we have fewer initial dimensions. It would be much too burdensome to try to interpret all of these components. 

### Investigating loadings

In order to understand a PCA model, we typically look at the first $N$ components to try to understand which aspects of the data are being used to explain variation. The loadings on the components allow us to do this, but looking at how the sales of each product influence the linear combination represented by each component. Remember: values that are large in absolute value play the greatest role. Additionally, values of different signs indicate the spectrum of the component. Typically we'll see both positive and negative values, indicating a compenent is contrasting between two different types of observations. 

This plotting code below is found from [this helpful blog post](https://medium.com/analytics-vidhya/pca-and-how-to-interpret-it-with-python-8aa664f7a69a).

In [ ]:
loadings = pd.DataFrame(pca.components_, columns=prod_sales.columns[1:])
loadings.transpose().sort_values(0)

This display isn't the most helpful, but here we're seeing how each product loads on each component. We've sorted by the first component (number zero). We can see that the smallest loading is positive but close to zero. The largest loading, chicken breasts, has a weight of 0.25. Let's take a closer look at this component, plotting the 20 largest loadings.   

In [ ]:
pc_loading = loadings.loc[0, :].sort_values(0).tail(n=20)
pc_loading.plot.bar()
plt.ylabel("PC1 Loading")
plt.tight_layout()
plt.show()


This component has positive loadings for every product and has the highest weights on the most popular product. Essentially, this component measures how much the owner spends at the Wedge. 

Let's take a look at the second component. Before we do that, let's write a function that will plot the component for us. This was developed from the blog post mentioned above and took a lot of trial and error to get right.  


In [ ]:
def show_component(component,cutoff=0.05) : 
    
    pc_loading = loadings.transpose().iloc[:,component-1]
    pc_loading = pc_loading.loc[pc_loading.abs() > cutoff].sort_values()
    max_pc = max(pc_loading.abs())
    colors = ['C0' if l > 0 else 'C1' for l in pc_loading]

    pc_loading.plot.bar(color = colors)
    ax = plt.gca()
    plt.axhline(y=0,c="#888888")
    ax.set_ylim(-max_pc, max_pc)
    plt.tight_layout()
    plt.xticks(rotation=45, ha='right')
    plt.show()

    

In [ ]:
show_component(2,0.05)

In [ ]:
show_component(3,0.06)

In [ ]:
show_component(4,0.08)

In [ ]:
show_component(5,0.06)

In class we'll spend some time discussing these loadings. The R example (which is the solution to the old PCA assignment) dives deeper into these. 